## Import Libraries

In [1]:
import os
from crewai import Agent, Task, Crew
from crewai.flow.flow import Flow, start, listen  
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import DuckDuckGoSearchRun



## Load the  LLM


In [2]:
import os
from langchain_community.chat_models.llamacpp import ChatLlamaCpp

llm = ChatLlamaCpp(
    model_path=os.path.abspath("tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"),
    n_ctx=1024,
    n_threads=4,
    verbose=False
)

response = llm.invoke([("human", "Explain LangChain in one line.")])
print(response)





llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (1024) < n_ctx_train (2048) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (n

content='LangChain is a free, open-source translation memory and translation software for multiple languages.' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run--a7f767e9-baee-438e-9cca-94daddeb02d3-0'


## Load Tools

In [ ]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGo Search")
def search_tool(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    return DuckDuckGoSearchRun().run(query)



## Agent Definitions



In [17]:
loader_agent = Agent(
    role="Loader Agent",
    goal="Search and retrieve electric consumption data for Indian states (2018–2023).",
    backstory="A research assistant who finds credible data sources from the web.",
    tools=[search_tool],
    llm=llm,
    verbose=True
)

In [29]:
from langchain_experimental.tools import PythonREPLTool


@tool("DuckDuckGo Search")
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    return PythonREPLTool().run(query)


In [30]:
analyzer_agent = Agent(
    role="Analyzer Agent",
    goal="Analyze the CSV to find top 3 states with highest average consumption.",
    backstory="A data analyst who uses pandas to extract patterns.",
    tools=[search_tool1],
    llm=llm,
    verbose=True
)
#This one loads CSV and analyzes with pandas.

In [ ]:
visualizer_agent = Agent(
    role="Visualizer Agent",
    goal="Create bar charts to show electricity consumption per year by state.",
    backstory="A matplotlib expert for Indian state electricity data.",
    tools=[python_tool],
    llm=llm,
    verbose=True
)
#Makes visual plots of electricity data.

In [ ]:
summarizer_agent = Agent(
    role="Summarizer Agent",
    goal="Write a 2-3 sentence summary about which states consumed the most electricity and why.",
    backstory="A professional executive summary writer.",
    tools=[],
    llm=llm,
    verbose=True
)
#Pure LLM-based summary writer.



## Task Definitions

In [ ]:
load_task = Task(
    description="Search for electricity consumption datasets for Indian states (2018–2023). Focus on government sources. Print top 3 links.",
    agent=loader_agent
)

analyze_task = Task(
    description="Load a CSV file, calculate average consumption, and print top 3 states.",
    agent=analyzer_agent
)

visualize_task = Task(
    description="Use matplotlib/seaborn to plot electricity consumption for each state.",
    agent=visualizer_agent
)

summarize_task = Task(
    description="Write a summary of insights from the data and visualizations in 2–3 sentences.",
    agent=summarizer_agent
)
#Each task explains what the agent should do.

## Flow Definition

In [ ]:
flow = Sequential(
    steps=[
        load_task,
        Parallel(steps=[analyze_task, visualize_task, summarize_task])
    ]
)
#First, data is loaded. Then 3 tasks run in parallel.

##  Run and Show Result

In [ ]:
crew = Crew(flow=flow, verbose=True)
result = crew.kickoff()

print("\n Final Result:ssssss
